# Estimating covariance matrix on SkySim5000 dark matter halo catalog

In [ ]:
from IPython.display import Markdown
import numpy as np
import pyccl as ccl
import matplotlib.pyplot as plt
import scipy.integrate
import astropy.units as u
from astropy.table import Table
import GCRCatalogs

%matplotlib inline

In [ ]:
skysim_cat = GCRCatalogs.load_catalog('skysim5000_v1.1.1_small')

In [ ]:
cosmo_ss  = skysim_cat.cosmology
cosmo_ss

## Extract DM haloes from the catalog in a given mass and redshift range. 

In [ ]:
%%time
mmin = 1.e12 # Msun (M_fof)
zmin = 0.
zmax = 1.0
dm_halos = Table(skysim_cat.get_quantities(['halo_mass','hostHaloMass','redshift','ra', 'dec', 'halo_id',
                                             'baseDC2/sod_halo_mass'],
                                            filters=[f'halo_mass > {mmin}','is_central==True',
                                            f'redshift>{zmin}', f'redshift<{zmax}']))
dm_halos['m200c'] = dm_halos['baseDC2/sod_halo_mass']/cosmo_ss.h

In [ ]:
dm_halos = Table(dm_halos[dm_halos['m200c'] > 0])

In [ ]:
print(min(np.log10(dm_halos['m200c'])), max(np.log10(dm_halos['m200c'])))

In [ ]:
z_corner = np.linspace(0.25, 1, 10)
logm_corner = np.linspace(13, 14.5, 10)

In [ ]:
type(dm_halos)

In [ ]:
import healpy
healpix = healpy.ang2pix(2**8, np.array(dm_halos['ra']), np.array(dm_halos['dec']), nest=True, lonlat=True)
healpix

# Estimate the covariance matrix for binned cluster count

In [ ]:
import CL_COUNT_DATAOPS_cluster_abundance_covariance as covariance

In [ ]:
Covariance = covariance.Covariance_matrix()

## Bootstrap

In [ ]:
Covariance.compute_boostrap_covariance(catalog = dm_halos, proxy_colname = 'm200c', 
                                    redshift_colname = 'redshift', proxy_corner = 10**logm_corner, 
                                    z_corner = z_corner, n_boot = 100)

In [ ]:
plt.imshow(np.corrcoef(Covariance.Bootstrap_covariance_matrix))

## Jackknife

In [ ]:
Covariance.compute_jackknife_covariance_healpy(catalog = dm_halos, proxy_colname = 'm200c', 
                                           redshift_colname = 'redshift',z_corner = z_corner, 
                                           proxy_corner = 10**logm_corner, ra_colname = 'ra', 
                                           dec_colname = 'dec', n_power = 8, N_delete = 1)

In [ ]:
plt.imshow(np.corrcoef(Covariance.Jackknife_covariance_matrix))